In [1]:
import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#import imageclassfunctions as icf
import json
import argparse

In [2]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):

    checkpoint = torch.load(filepath, map_location='cpu')

    if checkpoint['model name'] == "vgg13":
        model = models.vgg13(pretrained=True)
    else:
        model = models.vgg19(pretrained=True)

    from collections import OrderedDict
    classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, checkpoint['Hidden units'])),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(checkpoint['Hidden units'], 20)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    model.classifier = classifier
    model.load_state_dict(checkpoint['state_dict'])
    
    return model   

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    from PIL import Image
    
    im = Image.open(image)
    w = im.width
    h = im.height

    if w <= h:
        h = int(256 * h / w)
        w = 256
    else:
        w = int(256 * w / h)
        h = 256
        
    im2 = im.resize((w, h))
    im3 = im2.crop((int(w/2)-112, int(h/2)-112, int(w/2)+112, int(h/2)+112))
    np_image = ((np.array(im3) / 255 - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]) .transpose(2,0,1)
    
    return np_image


def predt(image_path, model, device, topk):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    checkimg = process_image(image_path)
        
    imgt = torch.from_numpy(checkimg)
    imgt = imgt.type(torch.FloatTensor)
    
    imgt.unsqueeze_(0)
    
    preResult = model(imgt)
    return preResult.topk(topk)

In [3]:

probs, fcla_ids = predt('yellowiris01.jpg', load_checkpoint('checkpoint_3.pth'), 'cpu', 3)

pro_n = probs.detach().numpy()[0]
fcla_n = fcla_ids.numpy()[0] 

pro_n = np.exp(pro_n)
print(pro_n)

foldernum = list(range(20))
folderind = [1, 10] + list(range(11, 20)) + [2, 20] + [3] + [4] + [5] + [6] + [7] + [8] + [9]

id_to_foldernum = dict(zip(foldernum, folderind))

fclass = []
for fcla_nn in fcla_n:
    fclass.append(id_to_foldernum[fcla_nn])
print(fclass)

fclass = np.array(fclass)
fclastr = fclass.astype(np.str)
fnames = []

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
for fstr in fclastr:
    fnames.append(cat_to_name[fstr]) 
print(fnames)


[0.99028665 0.00526081 0.00394751]
[15, 12, 16]
['yellow iris', "colt's foot", 'globe-flower']
